# Notebook for reading TGraphs from '.C' files

With this notebook you can read TGraphs from '.C' files. As explained in the "Getting started" notebook, a `Submission` needs to exist or be created.

In [1]:
from hepdata_lib import Submission
submission = Submission()

Welcome to JupyROOT 6.20/04


In [2]:
from hepdata_lib import Table
table = Table("MyLimit")
table.description = "Here is the description."

Here we import the `CFileReader` class from `hepdata_lib` and read a '.C' file.

In [3]:
from hepdata_lib.c_file_reader import CFileReader
c_file = "example_inputs/brazilianFlag_BulkZZ_VVnew_new_combined_13TeV.C"
reader = CFileReader(c_file)
graphs = reader.get_graphs()

/Users/clange/hepdata_lib/hepdata_lib/root_utils.py:412: FutureWarning: ROOT.Double is deprecated and will disappear in a future version of ROOT. Instead, use ctypes.c_double for pass-by-ref of doubles
  graph.GetPoint(i, x_val, y_val)


Once you have read the '.C' file, you can list the included graphs with the following command:

In [4]:
list(graphs.keys())

['Graph0', 'Graph1', 'BulkZZ_grshade', 'Graph2', 'Graph3', 'BulkZZ_gtheory']

`CFileReader` lists all `TGraphs` as a dictionary. You can access the included values by opening individual dictionaries and choosing the variable you want.

In [5]:
from hepdata_lib import Variable, Uncertainty
d = Variable("Bulk graviton mass", is_independent=True, is_binned=False, units="GeV")
d.values = graphs["Graph0"]['x'][:30]

obs = Variable("BulkZZ", is_independent=False, is_binned=False, units="pb")
obs.values = graphs["Graph2"]['y'][:30]
obs.add_qualifier("Limit", "Observed")
obs.add_qualifier("SQRT(S)", 13, "TeV")
obs.add_qualifier("LUMINOSITY", 35.9, "fb$^{-1}$")

exp = Variable("BulkZZ", is_independent=False, is_binned=False, units="pb")
exp.values = graphs["Graph3"]['y'][:30]
exp.add_qualifier("Limit", "Expected")
exp.add_qualifier("SQRT(S)", 13, "TeV")
exp.add_qualifier("LUMINOSITY", 35.9, "fb$^{-1}$")

#+/- 1 sigma
unc_1s = Uncertainty("1 s.d.", is_symmetric=False)
unc_1s.set_values_from_intervals(zip(graphs["Graph1"]['y'][:30], graphs["Graph1"]['y'][::-1][:30]), nominal=exp.values)
exp.add_uncertainty(unc_1s)

# +/- 2 sigma
unc_2s = Uncertainty("2 s.d.", is_symmetric=False)
unc_2s.set_values_from_intervals(zip(graphs["Graph0"]['y'][:30], graphs["Graph0"]['y'][::-1][:30]), nominal=exp.values)
exp.add_uncertainty(unc_2s)

All that needs to be done is to add the variables to the `Table`, the `Table` to the `Submission`, and then run the command to create the output files. Again, please refer to the 'Getting started' notebook for a complete example.

In [6]:
table.add_variable(d)
table.add_variable(obs)
table.add_variable(exp)
submission.add_table(table)
submission.create_files("example_output")

In [7]:
!ls example_output

mylimit.yaml    submission.yaml


In [8]:
!head -n 30 example_output/mylimit.yaml

dependent_variables:
- header:
    name: BulkZZ
    units: pb
  qualifiers:
  - name: Limit
    value: Observed
  - name: SQRT(S)
    units: TeV
    value: 13
  - name: LUMINOSITY
    units: fb$^{-1}$
    value: 35.9
  values:
  - value: 0.033069
  - value: 0.023478
  - value: 0.017733
  - value: 0.013937
  - value: 0.01084
  - value: 0.0086169
  - value: 0.0069185
  - value: 0.0056197
  - value: 0.0046831
  - value: 0.0039838
  - value: 0.0034093
  - value: 0.0030346
  - value: 0.0027099
  - value: 0.0024539
  - value: 0.0022541
  - value: 0.0020543
